# Predicting Major Flight Delays

Our client, FlightChicken, is developing a service to help users predict whether their flight may experience a severe delay. They would like us to develop a model for this. Giving users a heads up about a potential travel disruption can be tremendously helpful. Severe delays can cause travelers to miss connecting flights, miss important events and more.

There are millions of flights each year across thousands of airports and airlines (major and minor). For this MVP FlightChicken is tasking us to develop a minimum viable product (MVP) that supports 8 of the biggest airlines and 62 major domestic airports.

## Business Understanding

Our task is to solve the following business problems:
* Define what constitutes a major delay
* Collect and clean data on US flight delays
* Use supervised learning to develop a model for detecting major delays

Our client asked to build an MVP that support major US airlines and airports. These will be defined as follows:

* [Top 8 US Airlines by market share](https://www.statista.com/statistics/250577/domestic-market-share-of-leading-us-airlines/)
 * American Airlines
 * Delta Air Lines
 * United Airlines
 * Southwest Airlines
 * Alaska Airlines
 * JetBlue Airways
 * Spirit
 * SkyWest
* [Large airport hubs](https://www.faa.gov/airports/planning_capacity/passenger_allcargo_stats/passenger/media/cy20-commercial-service-enplanements.pdf)
 * "The term hub is used by the FAA to identify very busy commercial service airports. Large hubs are the airports that each account for at least one percent of total U.S. passenger enplanements."
 * In 2020 these accounted for 84% of all enplanements

### What is a 'major delay'?

No one likes any delay, but there is a world of difference between a 5 minute and a 3 hour delay. So the first challenge is to understand what our target is.

The biggest and most common consequence of a delay is that it may cause you to miss a connecting flight. Therefore, we will use average layover times to define a "major" delay. In other words, if a delay is severe enough to cause a traveler to potentially miss their connecting flight, we will call that a "major delay".

Multiple sources ([1](https://travel-made-simple.com/layover-long-enough/), [2](https://www.alternativeairlines.com/dealing-with-a-short-layover), [3](https://www.mic.com/articles/192954/how-much-time-do-you-really-need-for-a-layover), [4](https://traveltips.usatoday.com/minimum-time-should-allow-layovers-109029.html)) advise travelers to allow at least 1 hour for a connecting flight. Therefore, we will use that as the cutoff. For this project, we define a "major delay" as an arrival delay of 1+ hours.

### What causes delays?

Before we dive into data, it would be good to know common delay causes.

The Bureau of Tranpostation Statistics (BTS) reports the following:

1. **Air Carrier Delay 41%** "Maintenance or crew problems, aircraft cleaning, baggage loading, fueling, etc."
2. **Aircraft Arriving Late 30%**  "A previous flight with same aircraft arrived late, causing the present flight to depart late."
3. **National Aviation System Delay 22%** Things such as "non-extreme weather conditions, airport operations, heavy traffic volume, and air traffic control."
4. **Security Delay 0.2%** "Eacuation of a terminal or concourse, re-boarding of aircraft because of security breach..."
5. **Extreme Weather 7%** "Significant meteorological conditions...such as tornado, blizzard or hurricane."

The BTS notes that the above categorization system makes it seem like weather delays aren't a significant cause. That's because only extreme weather is coded seperately. For example, weather can be what causes an airplane to arrive late or be what is causing delays tagged 'National Aviation System Delay'. **The BTS attributes weather causes to 27% of delay minutes in 2020.**

### Importing Relevant Libraries
First, we'll bring in some libraries that will make it easier for us to work with our data.

In [1]:
import pandas as pd
import glob
import os
import requests
import json
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
import time

from datetime import datetime, timedelta
from pandas import Timestamp

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

import airportsdata
from pytz import timezone
import pytz

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

## Data

To complete this project, we will be using data from several sources.

1. **Bureau of Transportation Statistics: Carrier On-Time Performence Database.** This database contains scheduled and actual departure and arrival times reported by certified U.S. air carriers that account for at least one percent of domestic scheduled passenger revenues. The data is collected by the Office of Airline Information, Bureau of Transportation Statistics (BTS).
2. **weatherAPI** Because weather is such a contributing factor to delays, we'll connect to a weather API service to pull in weather conditions for each flight. This includes information on: average visibility, precipitation, temperatures and wind speeds.
3. **Additional information on airports from the 'airportsdata' library :**  We installed and imported airports data which contains data inside json documents one each airport. This data includes things like: elevation, latitude, longitude, and timezone.

### Data Preperation & Cleaning

#### Carrier On-Time Performence Database
This data can only be downloaded by month, which means it is split among many files. Data is downloaded from [this page](https://www.transtats.bts.gov/DL_SelectFields.aspx?gnoyr_VQ=FGK&QO_fu146_anzr=b0-gvzr).

In [7]:
flight_data = glob.glob(os.path.join('data/downloaded/carrier-on-time-performence', "*.csv"))
carrier_data = pd.concat((pd.read_csv(f) for f in flight_data), ignore_index=True)

In [8]:
# Let's inpsct the data
carrier_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7580724 entries, 0 to 7580723
Data columns (total 25 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY_OF_MONTH         int64  
 3   DAY_OF_WEEK          int64  
 4   FL_DATE              object 
 5   MKT_CARRIER          object 
 6   MKT_CARRIER_FL_NUM   int64  
 7   OP_CARRIER           object 
 8   TAIL_NUM             object 
 9   OP_CARRIER_FL_NUM    int64  
 10  ORIGIN               object 
 11  DEST                 object 
 12  CRS_DEP_TIME         int64  
 13  DEP_DELAY            float64
 14  DEP_DELAY_NEW        float64
 15  CRS_ARR_TIME         int64  
 16  ARR_DELAY_NEW        float64
 17  CANCELLED            float64
 18  CRS_ELAPSED_TIME     float64
 19  DISTANCE             float64
 20  CARRIER_DELAY        float64
 21  WEATHER_DELAY        float64
 22  NAS_DELAY            float64
 23  SECURITY_DELAY       float64
 24

In [9]:
carrier_data.head()

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,MKT_CARRIER,MKT_CARRIER_FL_NUM,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_DELAY,DEP_DELAY_NEW,CRS_ARR_TIME,ARR_DELAY_NEW,CANCELLED,CRS_ELAPSED_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2021,7,13,2,7/13/2021 12:00:00 AM,WN,2268,WN,N8627B,2268,SAN,DEN,1825,-5.0,0.0,2140,7.0,0.0,135.0,853.0,NaN,NaN,NaN,NaN,NaN
1,2021,7,13,2,7/13/2021 12:00:00 AM,WN,3815,WN,N231WN,3815,SAN,DEN,1505,NaN,NaN,1825,NaN,1.0,140.0,853.0,NaN,NaN,NaN,NaN,NaN
2,2021,7,13,2,7/13/2021 12:00:00 AM,WN,5013,WN,N8745K,5013,SAN,DEN,1210,24.0,24.0,1530,18.0,0.0,140.0,853.0,18.0,0.0,0.0,0.0,0.0
3,2021,7,13,2,7/13/2021 12:00:00 AM,WN,5306,WN,N8728Q,5306,SAN,DEN,2050,79.0,79.0,5,77.0,0.0,135.0,853.0,59.0,0.0,0.0,0.0,18.0
4,2021,7,13,2,7/13/2021 12:00:00 AM,WN,1111,WN,N8749Q,1111,SAN,HNL,1840,1.0,1.0,2140,0.0,0.0,360.0,2614.0,NaN,NaN,NaN,NaN,NaN


Looks like we have over 7 million records. Below is a dictionary that explains what each column means:

| **Feature**         | **Description**                                                                                                                                                 |
|---------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------|
| YEAR                | Year flight took place                                                                                                                                          |
| MONTH               | Month flight took place                                                                                                                                         |
| DAY_OF_MONTH        | Day of the month flight took place                                                                                                                              |
| DAY_OF_WEEK         | Day of the week flight took place (1=Monday, 2 = Tuesday, etc.)                                                                                                 |
| FL_DATE             | Flight Date (yyyymmdd)                                                                                                                                          |
| MKT_CARRIER         | Code assigned by IATA and commonly used to identify a carrier.                                                                                                  |
| MKT_CARRIER_FL_NUM  | Flight Number                                                                                                                                                   |
| OP_CARRIER          | Code assigned by IATA and commonly used to identify a carrier. This one applies to the airline operating a flight (not always the flight that sold the tickets) |
| TAIL_NUM            | Tail number of plane assigned for the flight (unique to each aircraft)                                                                                          |
| OP_CARRIER_FL_NUM   | Flight number for the operating airline                                                                                                                         |
| ORIGIN              | Airport where the flight originated                                                                                                                             |
| DEST                | Destination airport                                                                                                                                             |
| CRS_DEP_TIME        | Scheduled departure time                                                                                                                                        |
| DEP_DELAY           | Departure delay in minutes. Early departures are represented by negative values.                                                                                |
| DEP_DELAY_NEW       | Departure delay in minutes. Early departures are represented by 0.                                                                                              |
| CRS_ARR_TIME        | Arrival delay in minutes. Early arrivals are represented by negative values.                                                                                    |
| ARR_DELAY_NEW       | Arrival delay in minutes. Early arrivals are represented by 0.                                                                                                  |
| CANCELLED           | Cancelled Flight Indicator (1=Yes)                                                                                                                              |
| CRS_ELAPSED_TIME    | Scheduled time of flight in minutes (flight duration in minutes)                                                                                                |
| DISTANCE            | Distance between airports (miles)                                                                                                                               |
| CARRIER_DELAY       | Carrier Delay, in Minutes	                                                                                                                                       |
| WEATHER_DELAY       | Weather Delay, in Minutes	                                                                                                                                       |
| NAS_DELAY           | National Air System Delay, in Minutes                                                                                                                           |
| SECURITY_DELAY      | Security Delay, in Minutes	                                                                                                                                      |
| LATE_AIRCRAFT_DELAY | Late Aircraft Delay, in Minutes	                                                                                                                                 |

Based on the above, we will use 'ARR_DELAY_NEW' as the field to calculate our target. This is because we want to detect whether a flight arrives 60 minutes late or more.

Before we get to that, we will clean up our data a bit more.

In [10]:
# First, we drop cancelled flights from our dataset. While interesting
# we are interested in detecting delayed flights not cancelled ones
carrier_data = carrier_data.loc[carrier_data['CANCELLED'] == 0]
carrier_data.drop(['CANCELLED'], axis = 1, inplace=True)

Next, we look for missing values.

In [11]:
# Check how many missing values are in each column:
carrier_data.isna().sum()

YEAR                         0
MONTH                        0
DAY_OF_MONTH                 0
DAY_OF_WEEK                  0
FL_DATE                      0
MKT_CARRIER                  0
MKT_CARRIER_FL_NUM           0
OP_CARRIER                   0
TAIL_NUM                     0
OP_CARRIER_FL_NUM            0
ORIGIN                       0
DEST                         0
CRS_DEP_TIME                 0
DEP_DELAY                    0
DEP_DELAY_NEW                0
CRS_ARR_TIME                 0
ARR_DELAY_NEW            19033
CRS_ELAPSED_TIME             0
DISTANCE                     0
CARRIER_DELAY          5864061
WEATHER_DELAY          5864061
NAS_DELAY              5864061
SECURITY_DELAY         5864061
LATE_AIRCRAFT_DELAY    5864061
dtype: int64

Looks like there are some missing values in the arrival delay field and a lot of missing values in the delay cause fields.
Here is how we handle these missing values:

1. For missing ARR_DELAY_NEW field: we can simply drop these records since they account for just 0.25% of our entire data.
2. Missing delay values: while it would be interesting to have these fields, it's clear that they are faulty. Luckily, we don't need to know delay causes in modeling (since it's data that is describing something after the fact). Because of that, we will just drop these columns altogether.

In [12]:
# First we drop the delay cause columns
cols_to_drop = ['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']
carrier_data.drop(cols_to_drop, axis = 1, inplace=True)

In [13]:
# Next we drop the 0.25% of records that are missing an arrival delay
carrier_data = carrier_data[np.isfinite(carrier_data['ARR_DELAY_NEW'])]

### Engineering the Target

As stated above, we are interested in detecting delays greater than 60 minutes. We will create a field that each record as experiencing a major delay or not.

In [14]:
# First, we create a function to label our data
delay_conditions = [
    (carrier_data['ARR_DELAY_NEW'] >= 60),
    (carrier_data['ARR_DELAY_NEW'] < 60),
]
delay_values = ['Major Delay', 'No Major Delay']  

carrier_data['target'] = np.select(delay_conditions, delay_values)

Let's take a look at how many major delays are in our data.

In [16]:
carrier_data['target'].value_counts(normalize=True)

No Major Delay    0.930351
Major Delay       0.069649
Name: target, dtype: float64

So, it looks like we have an unbalanced dataset with just 7% of our data having a major delay.

### Bringing In Additional Data from the airportsdata library
Next, we'll use the airportsdata package to load in some additional details for each airport (like its location, timezone, and elevation).

In [17]:
# first we load the data fromt he library
airports = airportsdata.load('IATA')
# Let's take a look at what that data looks like
airports

{'ICY': {'icao': '19AK',
  'iata': 'ICY',
  'name': 'Icy Bay Airport',
  'city': 'Icy Bay',
  'subd': 'Alaska',
  'country': 'US',
  'elevation': 50.0,
  'lat': 59.96900177,
  'lon': -141.662002563,
  'tz': 'America/Anchorage'},
 'HGZ': {'icao': '2AK6',
  'iata': 'HGZ',
  'name': 'Hog River Airport',
  'city': 'Hogatza',
  'subd': 'Alaska',
  'country': 'US',
  'elevation': 534.0,
  'lat': 66.2161026,
  'lon': -155.6690063,
  'tz': 'America/Anchorage'},
 'BYW': {'icao': '38WA',
  'iata': 'BYW',
  'name': 'Blakely Island Airport',
  'city': 'Blakely Island',
  'subd': 'Washington',
  'country': 'US',
  'elevation': 66.0,
  'lat': 48.5789985657,
  'lon': -122.825996399,
  'tz': 'America/Los_Angeles'},
 'BDF': {'icao': '3IS8',
  'iata': 'BDF',
  'name': 'Rinkenberger Restricted Landing Area',
  'city': 'Bradford',
  'subd': 'Illinois',
  'country': 'US',
  'elevation': 808.0,
  'lat': 41.2308998108,
  'lon': -89.6156997681,
  'tz': 'America/Chicago'},
 'BCS': {'icao': '65LA',
  'iata': 'B

The airports data is a dictionary of dictionaries. The first key is the Airport IATA code. Within each dictionary we have data on that particular airport, such as its name, city, country, elevation and more.

We will create a dataframe containing all this information and then merge it into our carrier DataFrame.

In [18]:
# First, we initiate some empty lists to store our data
airports_list = []
lat_list = []
lon_list = []
elevation_list = []
tz_list = []

# Now we will loop through the keys (airports) in the data and pull out relevant information
# In this case, we are mostly interested in latitude, longitude, elevation and timezone
for key in airports.keys():
    airport = airports[key]['iata']
    lat = airports[key]['lat']
    lon = airports[key]['lon']
    elevation = airports[key]['elevation']
    tz = airports[key]['tz']
    airports_list.append(airport)
    lat_list.append(lat)
    lon_list.append(lon)
    elevation_list.append(elevation)
    tz_list.append(tz)

# Now we createa  dataframe using the data we just pulled
airport_locations = pd.DataFrame(
    {'Airport': airports_list,
     'Latitude': lat_list,
     'Longitude': lon_list,
     'Timezone': tz_list,
     'Elevation': elevation_list,
    })

# Before moving on, we also create an additional feature that combines the latitude and longitude
airport_locations['lat-long'] = airport_locations['Latitude'].astype(str) + ',' + airport_locations['Longitude'].astype(str)

Let's takea  quick look at what our data looks like.

In [19]:
airport_locations.head()

,Airport,Latitude,Longitude,Timezone,Elevation,lat-long
0,ICY,59.969002,-141.662003,America/Anchorage,50.0,"59.96900177,-141.662002563"
1,HGZ,66.216103,-155.669006,America/Anchorage,534.0,"66.2161026,-155.6690063"
2,BYW,48.578999,-122.825996,America/Los_Angeles,66.0,"48.5789985657,-122.825996399"
3,BDF,41.230900,-89.615700,America/Chicago,808.0,"41.2308998108,-89.6156997681"
4,BCS,29.866100,-90.022202,America/Chicago,0.0,"29.8661003113,-90.0222015381"


Great. Now we want to merge that data with our carrier_data. We want to do this for both the origin airport and the destination airport. We will use the Airport code as our key.

In [20]:
# We use the map method to bring in the first set of data
carrier_data['origin-lat-long'] = carrier_data['ORIGIN'].map(airport_locations.set_index('Airport')['lat-long'])
carrier_data['origin-tz'] = carrier_data['ORIGIN'].map(airport_locations.set_index('Airport')['Timezone'])
carrier_data['origin-elevation'] = carrier_data['ORIGIN'].map(airport_locations.set_index('Airport')['Elevation'])

# And now we repeat it for the destination
carrier_data['dest-lat-long'] = carrier_data['DEST'].map(airport_locations.set_index('Airport')['lat-long'])
carrier_data['dest-tz'] = carrier_data['DEST'].map(airport_locations.set_index('Airport')['Timezone'])
carrier_data['dest-elevation'] = carrier_data['DEST'].map(airport_locations.set_index('Airport')['Elevation'])

Next we want to check if our data was able to map on well.

In [21]:
# Check whether we were able to match all records
print('Missing origin timezones: {}'.format(carrier_data['origin-tz'].isna().sum()))
print('Missing origin elevations: {}'.format(carrier_data['origin-elevation'].isna().sum()))
print('Missing origin locations: {}'.format(carrier_data['origin-lat-long'].isna().sum()))
print('------------------')
print('Missing destination timezones: {}'.format(carrier_data['dest-tz'].isna().sum()))
print('Missing destination  elevations: {}'.format(carrier_data['dest-elevation'].isna().sum()))
print('Missing destination  locations: {}'.format(carrier_data['dest-lat-long'].isna().sum()))

Missing origin timezones: 13825
Missing origin elevations: 13825
Missing origin locations: 13825
------------------
Missing destination timezones: 13810
Missing destination  elevations: 13810
Missing destination  locations: 13810


Looks like about 13,000 records couldn't be matched. That's not bad considering we have over 7,000,000 record total. We'll simply drop the Null values.

In [22]:
# Dropping the few values we couldn't map succesfully
carrier_data = carrier_data.dropna(subset=['origin-tz', 'origin-elevation', 'origin-lat-long',
                                           'dest-tz', 'dest-elevation', 'dest-lat-long'])

#### Fixing time & date

Our time and date data is a bit messy.

In [23]:
carrier_data[['FL_DATE', 'CRS_DEP_TIME', 'CRS_ARR_TIME']]

,FL_DATE,CRS_DEP_TIME,CRS_ARR_TIME
0,7/13/2021 12:00:00 AM,1825,2140
2,7/13/2021 12:00:00 AM,1210,1530
3,7/13/2021 12:00:00 AM,2050,5
4,7/13/2021 12:00:00 AM,1840,2140
5,7/13/2021 12:00:00 AM,840,1145
...,...,...,...
7580719,1/9/2022 12:00:00 AM,1224,1352
7580720,1/10/2022 12:00:00 AM,1224,1352
7580721,1/11/2022 12:00:00 AM,1224,1352
7580722,1/12/2022 12:00:00 AM,1224,1352


The FL_DATE field doesn't have time in it (the 12:00:00AM is just a placeholder). CRS_DEP_TIME and CRS_ARR_TIME are floats. We also don't have the arrival date.

We'll work on fixing all those issues.

In [24]:
# First, we drop the FL_DATE placeholder time
carrier_data['FL_DATE'] = pd.to_datetime(carrier_data['FL_DATE']).dt.date

Next, let's clean up the CRS_DEP_TIME and CRS_ARR_TIME fields so they are expressed as actual time formats.

In [25]:
# To help us, we create a function to transform the data
def float_to_time(time):
    '''
    This function takes in a time expressed as a float, and returns a string with
    the time in a HH:MM format. For example, '5' becomes '00:05' and '151' becomes '01:51'
    '''
    time_str = str(time) # Take the float and transform it to a string
    digits = len(time_str) # Count the digits in the string so we know which transformation to use
    
    # Below are our transformations
    
    if digits < 2:
        return '00:0' + str(time)
    if digits == 2:
        return '00:' + str(time)
    if digits == 3:
        return '0' + time_str[:1] + ':' + time_str[1:]
    if digits == 4:
        return time_str[:2] + ':' + time_str[2:]

In [26]:
# Now we apply those transformations to our two time fields
carrier_data['CRS_DEP_TIME'] = carrier_data['CRS_DEP_TIME'].apply(float_to_time)
carrier_data['CRS_ARR_TIME'] = carrier_data['CRS_ARR_TIME'].apply(float_to_time)

# Next, we update the FL_DATE field so that it now contains the proper date AND time of takeoff
carrier_data['FL_DATE'] =  pd.to_datetime(carrier_data['FL_DATE'].astype(str) + ' ' + carrier_data['CRS_DEP_TIME'])

#### Creating an "Arrival Date" field
Creating this field is tricky. We can use the CRS_ELAPSED_TIME to help. If we add that field to the original flight date, the output will be the arrival time. However, the problem is that the arrival date would be in terms of the origin. So if a plane crosses timezones, we would have the wrong arrival time/date.

The solution we go with is to add timezone data to our FL_DATE field, add the flight duration to create a field that represents Arrival Date/Time relative to the Origin. Then we can use the 'destination timezone' field to conver that field to the destination timezone which will update the time and date appropriately.

In [27]:
# First we need to make our data, which is timezone naive, to timezone aware
carrier_data['FL_DATE'] = carrier_data['FL_DATE'].astype('datetime64[ns]')
carrier_data['FL_DATE'] = carrier_data.apply(lambda x: x['FL_DATE'].replace(tzinfo=timezone(x['origin-tz'])), axis=1)

Next, we want to know when the flight will be landing. We have the field CRS_ELAPSED_TIME to see how many minutes are supposed to take between departure and arrival. This is expressed in minutes. We can add that to the FL_DATE and then convert it to a local time.

In [28]:
carrier_data['flight_duration'] = pd.to_timedelta(carrier_data['CRS_ELAPSED_TIME'],'m')
carrier_data['FL_ARR_DATE_REL_ORIGIN'] = carrier_data['FL_DATE'] + carrier_data['flight_duration']
# And now we convert arrival time and date to a local time
carrier_data['FL_ARR_DATE'] = carrier_data.apply(lambda x: x['FL_ARR_DATE_REL_ORIGIN'].tz_convert(x['dest-tz']), axis=1)

C:\Users\Robert\anaconda3\envs\learn-env\lib\site-packages\pandas\core\arrays\datetimelike.py:1342: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized
  warnings.warn(


Lastly, we want to clean up what we did before. First, let's remove the timezone data from our date fields. We won't be needing it again, and having timezone aware values makes some calculations we will wanna use later not work. We can also drop the FL_ARR_DATE_REL_ORIGIN column since it will be useless.

In [29]:
# First we remove the timezone data
carrier_data['FL_DATE'] = carrier_data.apply(lambda x: x['FL_DATE'].replace(tzinfo=None), axis=1)
carrier_data['FL_ARR_DATE'] = carrier_data.apply(lambda x: x['FL_ARR_DATE'].replace(tzinfo=None), axis=1)

# Next we drop the unnecessary column
carrier_data.drop(columns=['FL_ARR_DATE_REL_ORIGIN', 'flight_duration'], inplace=True)

### Engineering Features
#### Airport Congestion
One hypothesis is that flight delays can be tied to airport "traffic" (or congestion). It stands to reason that if an airline has 100 flights scheduled to take off at 11am, there is a higher chance of delays than if there are only 10 flights scheduled to take off.

We'll create a set of features that put a number on this congestion.

First, we will define a time of day the flight is taking of (for example Early Morning). We'll then calculate average congestion during that time and day for each given airport.

In [30]:
# First, we want to know the hour of take off and landing
carrier_data['takeoff-hour'] = carrier_data['FL_DATE'].dt.hour.astype(int)
carrier_data['arriving-hour'] = carrier_data['FL_ARR_DATE'].dt.hour.astype(int)

Next, we want to create some set of conditions to determine the time off day each flight takes off and arrives.

We'll use the [Britannica Dictionary definitions](https://www.britannica.com/dictionary/eb/qa/parts-of-the-day-early-morning-late-morning-etc) for each time of day.

In [32]:
conditions = [
    (carrier_data['takeoff-hour'] >= 5) & (carrier_data['takeoff-hour'] <= 8),
    (carrier_data['takeoff-hour'] >= 8) & (carrier_data['takeoff-hour'] <= 12),
    (carrier_data['takeoff-hour'] > 12) & (carrier_data['takeoff-hour'] <= 15),
    (carrier_data['takeoff-hour'] > 15) & (carrier_data['takeoff-hour'] <= 17),
    (carrier_data['takeoff-hour'] > 17) & (carrier_data['takeoff-hour'] <= 19),
    (carrier_data['takeoff-hour'] > 19) & (carrier_data['takeoff-hour'] <= 21),
    (carrier_data['takeoff-hour'] >= 0) & (carrier_data['takeoff-hour'] < 5),
    (carrier_data['takeoff-hour'] > 21)
]
values = ['Early Morning', 'Late Morning', 'Early Afternoon', 'Late Afternoon',
          'Early Evening',  'Late Evening', 'Night', 'Night']

carrier_data['takeoff-time-of-day'] = np.select(conditions, values)

In [33]:
arr_conditions = [
    (carrier_data['arriving-hour'] >= 5) & (carrier_data['arriving-hour'] <= 8),
    (carrier_data['arriving-hour'] >= 8) & (carrier_data['arriving-hour'] <= 12),
    (carrier_data['arriving-hour'] > 12) & (carrier_data['arriving-hour'] <= 15),
    (carrier_data['arriving-hour'] > 15) & (carrier_data['arriving-hour'] <= 17),
    (carrier_data['arriving-hour'] > 17) & (carrier_data['arriving-hour'] <= 19),
    (carrier_data['arriving-hour'] > 19) & (carrier_data['arriving-hour'] <= 21),
    (carrier_data['arriving-hour'] >= 0) & (carrier_data['arriving-hour'] < 5),
    (carrier_data['arriving-hour'] > 21)
]

carrier_data['arrival-time-of-day'] = np.select(arr_conditions, values)

The next field we need is day of the week. We already have that for when the flight departs (the field is called DAY_OF_WEEK). We need that same field for when the flight arrives.

In [34]:
carrier_data['ARR_DAY_OF_WEEK'] = carrier_data['FL_ARR_DATE'].dt.dayofweek

In [35]:
# Let's take a look at what the output looks like
carrier_data[['ORIGIN', 'DEST', 'DAY_OF_WEEK', 'FL_DATE', 'FL_ARR_DATE', 'ARR_DAY_OF_WEEK']].head()

,ORIGIN,DEST,DAY_OF_WEEK,FL_DATE,FL_ARR_DATE,ARR_DAY_OF_WEEK
0,SAN,DEN,2,2021-07-13 18:25:00,2021-07-13 21:40:00,1
2,SAN,DEN,2,2021-07-13 12:10:00,2021-07-13 15:30:00,1
3,SAN,DEN,2,2021-07-13 20:50:00,2021-07-14 00:05:00,2
4,SAN,HNL,2,2021-07-13 18:40:00,2021-07-13 21:40:00,1
5,SAN,HNL,2,2021-07-13 08:40:00,2021-07-13 11:45:00,1


It looks like the method dt.dayofweek uses a different index than what was originally in our data. To make the two equal, we just need to add 1 to each value.

In [36]:
# Adjusting so day_of_week index match
carrier_data['ARR_DAY_OF_WEEK'] = carrier_data['ARR_DAY_OF_WEEK'] + 1

In [37]:
# Next we make the day_of_week columns more reader-friendly
day_of_week_translation = {1: 'Monday',
                          2: 'Tuesday',
                          3: 'Wednesday',
                          4: 'Thursday',
                          5: 'Friday',
                          6: 'Saturday',
                          7: 'Sunday'}

carrier_data['DAY_OF_WEEK'].replace(day_of_week_translation, inplace=True)
carrier_data['ARR_DAY_OF_WEEK'].replace(day_of_week_translation, inplace=True)

To calculate congestion, we will group and count our data for each airport, day of the week and time of day. To do that, we need to create a key.

In [38]:
# Takeoff Congestion Key
carrier_data['takeoff-congestion-key'] = carrier_data['ORIGIN'] \
                        + carrier_data['DAY_OF_WEEK'].astype(str) \
                        + carrier_data['takeoff-time-of-day']

# Arrival Congestion Key
carrier_data['arrival-congestion-key'] = carrier_data['DEST'] \
                        + carrier_data['ARR_DAY_OF_WEEK'].astype(str) \
                        + carrier_data['arrival-time-of-day']

In [40]:
# Now we create a new dataframe that holds data on congestion
# We can count the TAIL_NUM since that is a unique identifier for each plane
# In other words, it tells us how many unique planes were scheduled to take off/arrive at each unique key
airport_congestion_by_hour = carrier_data.groupby('takeoff-congestion-key')['TAIL_NUM'].count()
airport_congestion_by_hour = airport_congestion_by_hour.to_frame()
airport_congestion_by_hour.reset_index(inplace=True)
airport_congestion_by_hour.rename(columns={'TAIL_NUM': 'count_of_flights'}, inplace=True)

# We repeat the above for arriving flight congestion
airport_arrival_congestion_by_hour = carrier_data.groupby('arrival-congestion-key')['TAIL_NUM'].count()
airport_arrival_congestion_by_hour = airport_arrival_congestion_by_hour.to_frame()
airport_arrival_congestion_by_hour.reset_index(inplace=True)
airport_arrival_congestion_by_hour.rename(columns={'TAIL_NUM': 'count_of_flights_arriving'}, inplace=True)

The above calculated values are total airplane counts not average congestion. For example, the above would give us the total planes that departed JFK airport on sunday night in our dataset. Not how many planes depart on average PER sunday night at JFK. To get that value, we need to divide by the total number of records (Sunday nights at JFK) for each record.

In [42]:
# We count up the records
records = carrier_data.groupby('takeoff-congestion-key')['FL_DATE'].nunique().tolist()
arr_records = carrier_data.groupby('arrival-congestion-key')['FL_ARR_DATE'].nunique().tolist()

# Add the number of records to the congestion dataframe
airport_congestion_by_hour['num_records'] = records
airport_arrival_congestion_by_hour['num_arr_records'] = arr_records

# and finally we divide the total planes by the records to get an average congestion
airport_congestion_by_hour['avg-takeoff-congestion'] = airport_congestion_by_hour['count_of_flights'] / airport_congestion_by_hour['num_records']
airport_arrival_congestion_by_hour['avg-arrival-congestion'] = airport_arrival_congestion_by_hour['count_of_flights_arriving'] / airport_arrival_congestion_by_hour['num_arr_records']

In [43]:
# We can drop the columns we won't be using again
airport_congestion_by_hour.drop(columns=['count_of_flights', 'num_records'], inplace=True)
airport_arrival_congestion_by_hour.drop(columns=['count_of_flights_arriving', 'num_arr_records'], inplace=True)

Finally, we can combine the two dataframes into a singular congestion dataframe that has both arriving and departing flight congestion.

In [44]:
# And finally we can combine the two into a congestion dataframe
airport_congestion_by_hour = pd.merge(airport_congestion_by_hour, airport_arrival_congestion_by_hour,
                                      left_on='takeoff-congestion-key', right_on='arrival-congestion-key')
airport_congestion_by_hour.drop(columns=['arrival-congestion-key'], inplace=True)

airport_congestion_by_hour.rename(columns={"takeoff-congestion-key": "congestion-key"}, inplace=True)

# We save the file out
airport_congestion_by_hour.to_csv('data/prepared/airport_congestion.csv', index=False)

Finally, we can merge the congestion we just calculated into our main carrier_data dataframe.

In [45]:
# Now we add congestion data to our main dataframe
carrier_data = pd.merge(carrier_data, airport_congestion_by_hour, left_on='takeoff-congestion-key', right_on='congestion-key')
# updating key for merging in data at the destination airport
airport_congestion_by_hour = airport_congestion_by_hour.add_prefix('dest-')
# Now data on the congestion conditions of the airport where the flight is arriving
carrier_data = pd.merge(carrier_data, airport_congestion_by_hour, left_on='arrival-congestion-key', right_on='dest-congestion-key')

#### Proximity to Holidays
Another feature we want to engineer is whether a flight is happening close to a major holiday. And if its, how close to that holiday it is. The motivation behind this is the common knowledge that airport delays seem to get worse when a lot of people are travelling (which tends to happen around certain holidays).

In [47]:
# First, we can instatietae the calendar library which holds holidays and dates
cal = calendar()

# Let's define a date range which we want to pull up
date_range = pd.date_range(start='2021-01-01', end='2025-12-31')

# Next we can apply the 'holidays' method on our calendar to pull up US holidays between our start and end date
holidays = cal.holidays(start=date_range.min(), end=date_range.max(), return_name=True)
holidays.reset_index(name='holiday').rename(columns={'index':'date'})
holidays = holidays.to_frame()
holidays.reset_index(inplace=True)
holidays.columns = ['holiday_date', 'holiday_name']

# Save out the dataframe to a file on disk
# holidays.to_csv('data/prepared/holidays.csv', index=False)

In [48]:
# Now we are ready to merge those holiday details to our carrier_data dataframe
# We'll be using merge_asof, which requires our data to be sorted by date
carrier_data.sort_values('FL_DATE', inplace=True)

# Next we merge the data in with a tolerance of 7 days. That means we will take note of whether a holiday is within 7 days
# of a flight record (before or after)
carrier_data = pd.merge_asof(carrier_data, holidays, left_on='FL_DATE', right_on='holiday_date',
                       direction='nearest', tolerance=pd.Timedelta(days=7))

# Now we calculate the number of days between a flight record and the nearest holiday
# If a flight record is not within 7 days of a holiday we expect this to return NA
carrier_data['days-from-holiday'] = (carrier_data['FL_DATE'] - carrier_data['holiday_date']).dt.days

# If a flight happens EXACTLY on a holiday, let's pull in that holiday's name
carrier_data['holiday'] = carrier_data.loc[carrier_data['days-from-holiday'] == 0, 'holiday_name']
# If the result was an NA that means the flight record did not happen on a holiday. Let's record that
carrier_data['holiday'].fillna('Not a Holiday', inplace=True)

# Next we create a "distance from specific holiday" feature. That is, something that will tell us if a flight record is
# for example 4 days from Thanksgiving
carrier_data['days-from-holiday'] = carrier_data['days-from-holiday'].astype(str)
carrier_data['days-from-specific-holiday'] = carrier_data['holiday_name'] + '_' + carrier_data['days-from-holiday'].astype(str)

# Again, if the result was NA here, that means there was no holiday within 7 days of a flight record
carrier_data['days-from-specific-holiday'].fillna('no-close-holiday', inplace=True)

In [49]:
# Once again, we clean up columns we don't need anymore
carrier_data.drop(columns=['holiday_date', 'holiday_name'], inplace=True)

#### Time of day takoff & landing

Lastly, we'll engineer a continuous numeric feature to measure the time a flight takes off or lands.

The easiest way to do this is to create a feature that is a "distance from midnight", which we'll measure in minutes. I found this solution on [StackOverflow from user Pontus Hultkrantz](https://stackoverflow.com/questions/54787146/get-the-time-spent-since-midnight-in-dataframe).

In [51]:
carrier_data['takeoff-mins-from-midnight'] = ((pd.to_datetime(carrier_data['CRS_DEP_TIME'])
                                               - pd.to_datetime(carrier_data['CRS_DEP_TIME']).dt.normalize()) \
                                              / pd.Timedelta('1 minute')).astype(int)

# Midnight needs to be expressed as '00:00' not 24:00 so we change that
carrier_data['CRS_ARR_TIME'] = carrier_data['CRS_ARR_TIME'].replace({'24:00':'00:00'})

carrier_data['landing-mins-from-midnight'] = ((pd.to_datetime(carrier_data['CRS_ARR_TIME'])
                                               - pd.to_datetime(carrier_data['CRS_ARR_TIME']).dt.normalize()) \
                                              / pd.Timedelta('1 minute')).astype(int)

### Filtering for Airports & Airlines Relevant to Business Case
Our business case calls to build an MVP for major us airlines and and airports. We defined what those are at the top of this notebook.

So, we now move to filter down our dataset to only include those relevant airports and airlines.

These are taken from the FAA's designation of ['large hubs'](https://www.faa.gov/airports/planning_capacity/passenger_allcargo_stats/passenger/media/cy20-commercial-service-enplanements.pdf).


In [53]:
# Create list of relevant aiports based on business case
relevant_airports = ['ATL', 'DFW', 'DEN', 'ORD', 'LAX', 'CLT', 'LAS', 'PHX', 'MCO', 'SEA',
                     'MIA', 'IAH', 'JFK', 'FLL', 'EWR', 'SFO', 'MSP', 'DTW', 'BOS', 'SLC',
                     'PHL', 'BWI', 'TPA', 'SAN', 'MDW', 'LGA', 'BNA', 'IAD', 'DAL', 'DCA',
                     'PDX', 'AUS', 'HOU', 'HNL', 'STL', 'RSW', 'SMF', 'MSY', 'SJU', 'RDU',
                     'SJC', 'OAK', 'MCI', 'CLE', 'IND', 'SAT', 'SNA', 'PIT', 'CVG', 'CMH',
                     'PBI', 'JAX', 'MKE', 'ONT', 'ANC', 'BDL', 'OGG', 'BUR', 'OMA', 'MEM',
                     'BOI', 'RNO','CHS', 'OKC']

# Filter Dataframe to include only relevant airlines & airports
carrier_data = carrier_data[carrier_data['ORIGIN'].isin(relevant_airports)]

Before we further filter down the data to our relevant case, we'll save it out for graphing/vizualization purposes.

In [55]:
carrier_data.to_csv('data/prepared/data_for_graphing.csv', index=False)

Next, we want to filter for only flight records belonging to top airlines.

In [2]:
carrier_data = pd.read_csv('data/prepared/data_for_graphing.csv')

In [3]:
# Create list of relevant IATA airline designators based on business case
relevant_airlines = ['WN', # Southwest
                     'DL', # Delta
                     'OO', # SkyWest
                     'AA', # American Airlines
                     'UA', # United Airlines
                     'B6', # JetBlue
                     'AS', # Alaska Airlines
                     'NK', # Spirit Airlines
                    ]
carrier_data = carrier_data[carrier_data['MKT_CARRIER'].isin(relevant_airlines)]

#### Filtering for more frequently vistied locations
For the sake of efficiency, we can further filter down our data to include only destinations that are at least somewhat frequently traveled. We have millions of records, so we'll filter for destinations that had at least 1,000 flights to them.

In [4]:
carrier_data = carrier_data.groupby('DEST').filter(lambda x: len(x) > 1000)

### Merging in Weather Data
Lastly, we're ready to bring in weather data using the weatherAPI. To do this, we signed up for an account on [weatherapi.com](https://www.weatherapi.com/).

We're interested in using the API to pull weather data for the past year, since that is what we have flight data on. To replicate our code, you will need to use your own API key.

In [5]:
# First we define a start and end date that we will be pulling data for
start = datetime.strptime('2021-06-01', '%Y-%m-%d').date()
end = datetime.strptime('2022-07-31', '%Y-%m-%d').date()

# The API only allows us to pull up 30 days at a time
# That means we need to create buckets of 30 day date segments that we can iterate over

# Create a list that will hold all our dates
date_list = []

# Define a current date parameter that we will be increasing until we reach our end date
current_date = start

# and now we create a loop that goes through these dates

while current_date <= end:
    date_list.append(current_date)
    current_date_plus_30 = current_date + timedelta(days=30)
    date_list.append(current_date_plus_30)
    current_date = current_date_plus_30 + timedelta(days=1)

# We congvert these dates to a string
for i in range(0,len(date_list)):
    date_list[i] = date_list[i].strftime('%Y-%m-%d')
    
# Next we need to break up into two lists
# The API needs a start date and end date to pull up data for. So now we define these buckets

start_dates = []
end_dates = []

for i in range(0,len(date_list)):
    if i%2 == 0:
        start_dates.append(date_list[i])
    else:
        end_dates.append(date_list[i])

# Lastly, we also collect all locations we need to look up weather for
# To do that, we pull up the latitude-longitude field across our origins and destinations
# we then put them together in a single list

origins = list(carrier_data['origin-lat-long'].unique())
destinations = list(carrier_data['dest-lat-long'].unique())
lat_long_list = list(set(origins+destinations))

In [ ]:
# Create a function for pulling up our API key
def get_keys(path):
    '''
    Function takes a path to file containing json file
    It returns the file as a json document
    '''
    with open(path) as f:
        return json.load(f)

# Use that function to pull the file containing the weather API key. The below is the path on my computer
keys = get_keys("C:/Users/Robert/.secret/weather_api.json")

# Define the URL we will be hitting in the weather API
history_data_url = 'http://api.weatherapi.com/v1/history.json'

# Assign the API key to a variable
api_key = keys['api_key']

# Create a set of empty lists that will hold the weather details we retreive from the API
weather_latlong = []
weather_dates = []
weather_maxtemp_c = []
weather_mintemp_c = []
weather_avgtemp_c = []
weather_totalprecip_mm = []
weather_avgvis_km = []
weather_maxwind_kph = []
weather_avghumidity = []

# Now we loop over all the latitude/longitude locations in our dataframe (both destinations and origins)
# and we further loop through all the dates we defined earlier

for location in lat_long_list:
    print('Working on {}'.format(location))
    for i in range(0, len(start_dates)):
        r = requests.get(history_data_url + '?key=' + api_key + '&q=' + location + '&dt=' + start_dates[i] + '&end_dt=' + end_dates[i])
        d = json.loads(r.text)
        for j in range(0,31):
            weather_latlong.append(location)
            weather_dates.append(d['forecast']['forecastday'][j]['date'])
            weather_maxtemp_c.append(d['forecast']['forecastday'][j]['day']['maxtemp_c'])
            weather_mintemp_c.append(d['forecast']['forecastday'][j]['day']['mintemp_c'])
            weather_avgtemp_c.append(d['forecast']['forecastday'][j]['day']['avgtemp_c'])
            weather_totalprecip_mm.append(d['forecast']['forecastday'][j]['day']['totalprecip_mm'])
            weather_avgvis_km.append(d['forecast']['forecastday'][j]['day']['avgvis_km'])
            weather_maxwind_kph.append(d['forecast']['forecastday'][j]['day']['maxwind_kph'])
            weather_avghumidity.append(d['forecast']['forecastday'][j]['day']['avghumidity'])

Working on 42.36429977,-71.00520325


In [ ]:
# We take the above lists we created and throw them into a weather dataframe
weather_df = pd.DataFrame(
    {'lat-long': weather_latlong,
     'date': weather_dates,
     'maxtemp': weather_maxtemp_c,
     'mintemp': weather_mintemp_c,
     'avgtemp': weather_avgtemp_c,
     'totalprecip': weather_totalprecip_mm,
     'avgvis': weather_avgvis_km,
     'maxwind': weather_maxwind_kph,
     'avghumidity': weather_avghumidity
    })

In [ ]:
#Save the weather data out to a csv so we don't have to re-run through the API every time
weather_df.to_csv('data/downloaded/weather-data.csv', index=False)

In [2]:
# Load the weather data from disk if you didn't run the API above
# weather_df = pd.read_csv('data/downloaded/weather-data.csv')

Now that we downlaoded the weather data from the API we're ready to merge it into our flight records.

In [ ]:
# First, we need to create keys for matching the weather data to locations and dates
# carrier_data['FL_DATE'] = pd.to_datetime(carrier_data['FL_DATE'])
# carrier_data['FL_ARR_DATE'] = pd.to_datetime(carrier_data['FL_ARR_DATE'])
# Create keys (a unique identifier for a location and date)
carrier_data['weather-key'] = carrier_data['origin-lat-long'] + carrier_data['FL_DATE'].dt.date.astype(str)
carrier_data['dest-weather-key'] = carrier_data['dest-lat-long'] + carrier_data['FL_ARR_DATE'].dt.date.astype(str)
weather_df['weather-key'] = weather_df['lat-long'] + weather_df['date']

In [ ]:
# Now we merge the data using that weather key
carrier_data = carrier_data.merge(weather_df, on='weather-key')
# We add a prefix to all weather columns to distinguish them from the other weather data we just matched
weather_df = weather_df.add_prefix('dest-')
# And now we merge the destination weather data
carrier_data = carrier_data.merge(weather_df, on='dest-weather-key')

### Final checks
We run through some final checks on our data. That means checking for null values, and dropping columns that will not be relevant to our modeling efforts.

In [8]:
irrelevant_columns = ['YEAR', 'FL_DATE', 'MKT_CARRIER_FL_NUM', 'OP_CARRIER', 'TAIL_NUM',
                      'OP_CARRIER_FL_NUM', 'CRS_DEP_TIME', 'DEP_DELAY', 'DEP_DELAY_NEW', 'CRS_ARR_TIME',
                      'ARR_DELAY_NEW', 'origin-lat-long', 'origin-tz', 'dest-lat-long_x', 'dest-tz',
                      'flight_duration', 'FL_ARR_DATE_REL_ORIGIN', 'FL_ARR_DATE', 'takeoff-hour',
                      'arriving-hour', 'takeoff-congestion-key', 'arrival-congestion-key', 'congestion-key',
                      'dest-congestion-key', 'days-from-holiday', 'weather-key', 'dest-weather-key',
                      'lat-long', 'date', 'dest-lat-long_y', 'dest-date', 'dest-date', 'dest-lat-long_y',
                      'date', 'lat-long','dest-weather-key', 'weather-key']

carrier_data.drop(columns=irrelevant_columns, inplace=True)

## Saving out the cleaned data

We are done preparing all the data and features and are ready to start modeling prepping it for modeling. We save the data out and move on to our modeling notebook.

In [9]:
carrier_data.to_csv('data/prepared/data_for_modeling.csv', index=False)